In [33]:
#2019.10.18 准备写一个大盘分析程序，将相关数据显示在网页上，然后后台定时运行，分析每天结果，我想看时打开网页就行。
#2019.10.24 实现往前算指定天数ndays，将index_dailybasic中的总市值与每日指数中成交量、成交额合并一个DF显示

In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import tushare as ts
from datetime import date, datetime, timedelta
pro = ts.pro_api() 
#初始化tushare

In [35]:
#本段采集上证官网首页大盘信息，id=b,变量名前可以加b防重复。
import requests        #导入requests包
from bs4 import BeautifulSoup
url = 'http://www.sse.com.cn/'
strhtml = requests.get(url)        #Get方式获取网页数据
strhtml.encoding=strhtml.apparent_encoding
#requests会从服务器返回的响应头的 Content-Type 去获取字符集编码，如果content-type有charset字段那么requests才能正确识别编码，
#否则就使用默认的 ISO-8859-1. 一般那些不规范的页面往往有这样的问题. 
#requests的返回结果对象里有个apparent_encoding函数, apparent_encoding通过调用chardet.detect()来识别文本编码. 
#但是需要注意的是，这有些消耗计算资源
souphtml = BeautifulSoup(strhtml.text,'lxml')
#Beautiful Soup 可以从HTML或XML文件中提取数据,用来处理导航、搜索、修改分析树等功能。

b_a=souphtml.select('table')
#定位到显示大盘数据的表格，一共有三个table
b_d=[]
b_b=b_a[0].find_all("tr")
#取table0就是大盘数据，找到里面所有的行
for b_c in b_b:
    b_d.extend(b_c.text.split())
    #text是提取这些行的数据，split是分割数据，默认是以空格，换行符等进行分割
dname=b_d[::2]
#取出表名
dvalue=b_d[1::2]
#取出表名对应的表值
df1 = pd.DataFrame([dvalue], columns=dname) 
print(df1)

  上市公司/家 上市股票/只    总股本/亿股   流通股本/亿股     总市值/亿元    流通市值/亿元 平均市盈率/倍
0   1528   1571  39091.32  34745.16  334665.38  287984.09   14.06


In [36]:
#本段采集深证官网首页大盘信息，id=a,变量名前可以加a防重复。
opt = Options()
opt.add_argument('--headless')
# 创建chrome无界面对象
driver = webdriver.Chrome(options=opt)
#设置无界面
driver.get('http://www.szse.cn/')
#访问深证官网
time.sleep(3)
#停止等待，防止取出空数据
a_a=driver.find_elements_by_class_name('tab-pane')
#大盘情况放在class为tab-pane名字中，find_elements找出所有的tab_pane
a_c=[]
for a_b in a_a:
    if a_b.text!='':
        #一共能找到四个class，其中三个为空，要过滤掉
        a_c=a_b.text.split('\n') 
        #以\n分割字符串
table_name=a_c[::2]
#从0取值，步长2，取出列名
table_name.append('数据采集时间')
#增加列名：数据采集时间
table_value=a_c[1::2]
#从1取值，步长2，取出数值
a_d=driver.find_elements_by_class_name('tip')
#时间放在tip中，首页中一共有两个tip，取最后一个
a_f=[]
for a_e in a_d:
    a_f.append(a_e.text)
table_value.append(a_f[-1])
#对应增加采集到的日期
df2 = pd.DataFrame([table_value], columns=table_name) 
#df1深证官网的大盘数据

driver.quit()
print(df1)


  上市公司/家 上市股票/只    总股本/亿股   流通股本/亿股     总市值/亿元    流通市值/亿元 平均市盈率/倍
0   1528   1571  39091.32  34745.16  334665.38  287984.09   14.06


In [37]:
#2019.10.24 本段程序显示上证大盘情况。实现往前算指定天数ndays，将index_dailybasic中的总市值与每日指数中成交量、成交额合并一个DF显示
ndays=10
#ndays将来计划是由网页提交给数据库，可以由用户更改。
delta = timedelta(days=8)  
#timedelta可以实现日期时间直接相加，例子如下：
#now = datetime.now()
#day = date.today()
#n_days_after = now + delta  #当前日期推迟n天之后的时间
#n_days_forward  = now - delta  #当前日期向前推n天的时间
day=date.today()
#取今天的时间
end_date=(day-timedelta(days=1)).strftime('%Y%m%d')
#截止时间为前一天,#默认输出是2019-10-24的格式，用strftime换算成20191024
start_date=(day-timedelta(days=ndays)).strftime('%Y%m%d')
#开始时间根据ndays的不同，可以变化
dfa = pro.index_dailybasic(start_date=start_date,end_date=end_date,ts_code='000001.SH',fields='ts_code,trade_date,total_mv,turnover_rate')
#index_dailybasic提供大盘指数每日行情，目前主要是目前只提供上证综指，深证成指，上证50
#中证500，中小板指，创业板指的每日指标数据
df3 = pro.index_daily(ts_code='000001.SH',start_date=start_date,end_date=end_date,fields='ts_code,trade_date,close,change,pct_chg,vol,amount')
#指数每日行情，用主要输出参数close,open,high,low,pre_close
#change涨跌点相对昨天收价，pct_chg涨跌幅相对昨天收盘价
#vol成交量手，amount成交额千元
df3['total_mv']=dfa.total_mv
#将df1的total_mv合并df2
#dataframe增加一列用法，df['new']=[1,2,3,4] 
#dataframe增加一行用法，加loc,df.loc['new']=[1,2,3,4]
#--------------------我是分割线------------------------------------------------------------
#2019.10.19
#原本想将运行结果保存到数据库，然后用PHP读出来显示到网页上，今天突然想到PYTHON能不能直接保存结果为HTML？百度一下，居然发现DATAFRAME可以
#直接存为HTML文件，这下省事了。

print(df3)

     ts_code trade_date      close   change  pct_chg          vol  \
0  000001.SH   20191025  2954.9327  14.0112   0.4764  136860706.0   
1  000001.SH   20191024  2940.9215  -0.6973  -0.0237  126540290.0   
2  000001.SH   20191023  2941.6188 -12.7611  -0.4319  123812881.0   
3  000001.SH   20191022  2954.3799  14.7620   0.5022  117482048.0   
4  000001.SH   20191021  2939.6179   1.4766   0.0503  132475107.0   
5  000001.SH   20191018  2938.1413 -39.1929  -1.3164  149990678.0   
6  000001.SH   20191017  2977.3342  -1.3782  -0.0463  122527614.0   
7  000001.SH   20191016  2978.7124 -12.3335  -0.4123  149885496.0   

        amount      total_mv  
0  160224090.6  3.879876e+13  
1  141667233.0  3.864492e+13  
2  139653650.5  3.862913e+13  
3  130381741.2  3.877605e+13  
4  145685881.2  3.860444e+13  
5  162018272.3  3.852931e+13  
6  136051252.1  3.902802e+13  
7  166791549.4  3.904096e+13  


In [38]:
#2019.10.18 准备写一个大盘分析程序，将相关数据显示在网页上，然后后台定时运行，分析每天结果，我想看时打开网页就行。
#2019.10.24 本段程序显示深证大盘情况

dfa1 = pro.index_dailybasic(start_date=start_date,end_date=end_date,ts_code='399001.SZ',fields='ts_code,trade_date,total_mv,turnover_rate')
#index_dailybasic提供大盘指数每日行情，目前主要是目前只提供上证综指，深证成指，上证50
#中证500，中小板指，创业板指的每日指标数据
df4 = pro.index_daily(ts_code='399001.SZ',start_date=start_date,end_date=end_date,fields='ts_code,trade_date,close,change,pct_chg,vol,amount')
#指数每日行情，用主要输出参数close,open,high,low,pre_close
#change涨跌点相对昨天收价，pct_chg涨跌幅相对昨天收盘价
#vol成交量手，amount成交额千元
df4['total_mv']=dfa1.total_mv
#将df1的total_mv合并df2
#dataframe增加一列用法，df['new']=[1,2,3,4] 
#dataframe增加一行用法，加loc,df.loc['new']=[1,2,3,4]
#--------------------我是分割线------------------------------------------------------------
#2019.10.19
#原本想将运行结果保存到数据库，然后用PHP读出来显示到网页上，今天突然想到PYTHON能不能直接保存结果为HTML？百度一下，居然发现DATAFRAME可以
#直接存为HTML文件，这下省事了。
print(df4)



     ts_code trade_date      close    change  pct_chg          vol  \
0  399001.SZ   20191025  9660.4403  104.6790   1.0955  224961819.0   
1  399001.SZ   20191024  9555.7613  -11.9875  -0.1253  207222324.0   
2  399001.SZ   20191023  9567.7488  -74.3400  -0.7710  215975821.0   
3  399001.SZ   20191022  9642.0888   88.5159   0.9265  207552391.0   
4  399001.SZ   20191021  9553.5729   20.0677   0.2105  209913943.0   
5  399001.SZ   20191018  9533.5052 -111.8835  -1.1600  238042710.0   
6  399001.SZ   20191017  9645.3887    3.3292   0.0345  206596230.0   
7  399001.SZ   20191016  9642.0595  -29.6693  -0.3068  233148157.0   

        amount      total_mv  
0  245550683.1  1.502189e+13  
1  218608084.4  1.484785e+13  
2  228668699.0  1.486832e+13  
3  226621129.8  1.499340e+13  
4  224397554.7  1.486568e+13  
5  253797032.8  1.483966e+13  
6  225158089.2  1.501314e+13  
7  256657323.7  1.499648e+13  


In [39]:
#多个DATAFRAME写入同一个html的例子
import os
HEADER ='''<html>
            <head>
                <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
            </head>
            <body>
            '''
FOOTER = '''</body>
            </html>'''

 
with open(os.path.join(os.getcwd(), 'filesnew.html'), 'w',encoding='utf8') as f:
    f.write(HEADER)
    f.write('<h1><strong>' + '上证大盘综合情况' +'</strong></h1>')
    f.write(df1.to_html(escape=False,index=False,sparsify=True,border=1,index_names=True,header=True))
    f.write('<h1><strong>' + '深证大盘综合情况' +'</strong></h1>')
    f.write(df2.to_html(escape=False,index=False,sparsify=True,border=1,index_names=True,header=True))
    f.write('<h1><strong>' + '上证历史大盘综合情况' +'</strong></h1>')
    f.write(df3.to_html(escape=False,index=False,sparsify=True,border=1,index_names=True,header=True))
    f.write('<h1><strong>' + '深证历史大盘综合情况' +'</strong></h1>')
    f.write(df4.to_html(escape=False,index=False,sparsify=True,border=1,index_names=True,header=True))
    f.write(FOOTER)

